# Specification

- This is a command Line program using standard parameters, STDIN and STDOUT redirection
- Program name: missingMotif.py
- Input/Output: missingMotif.py < [input file] > [output file]

- --minMotif  minimum motif size to evaluate (int)
- --maxMotif maximum motif size to evaluate (int)
- --cutoff  Z-score cutoff (float)

Ex: missingMotif.py --minMotif=3 --maxMotif=8 --cutoff=-4.0 < [input file] > [output file]

# Background
As cutting edge genomic sequencing technologies mature, their accessibility is enhanced while costs shrink, in contrast to specialized technologies such as Single-molecule real-time (SMRT) that reveal methylomes. The application of methylomic data to predict R-M sites is established, however, costly overheads prevent adoption on the same scale as genomic sequencing technologies.

In contrast to established methods for R-M components,Stealth performs a statistical comparison between the genomic counts of motifs within a specified size range (k-mers), and probabilities based on a genome specific null model derived from a markov (k-2) chain. Stealth outputs a list of motifs that are systematically underrepresented in the genome, which are expected to include novel R-M substrate motifs. In a 2022 paper, Bernick showed that synthetic DNA designed to avoid some of these underrepresented motifs in H. Pylori yielded improvements in transformation efficiency of >10,000%.

# The signifigance of under-represented motifs

RM systems are typically composed of a suite of restriction nucleases and cognate methyltransferases. The restriction component of these RM systems provide a defense strategy by selectively cleaving DNA at recognition sites, often characterized by short 4-6 base pair reverse complement palindromic motifs. On the other hand, the modification component shields the host genome by enzymatically modifying potential cleavage sites within the host DNA. Meselson and Stahl (1958) demonstrated the semi-conservative model of bacterial double-stranded DNA genome replication [3] . This process yields chromosomal copies that are transiently hemi-methylated, as the nascent strand of the daughter chromosomes have not yet had the opportunity to undergo protective methylation by the RM system. What ensues is a race against time—a race between the RM cleavage machinery and the protective DNA methylation processes. This molecular interplay imparts a selective pressure on the genome towards reduction of endogenous RM recognition motifs, leading to the underrepresentation of RM motifs within the genome over time, allowing them to be identified as systemically underrepresented in genomic sequence.

Note: "Stealth" has recently been accepted for publication, and is now publicly available at: https://git.ucsc.edu/dbernick/stealth

# The following is my implimentation of a HMM based approach to identify such motifs


In [ ]:
#Example output
sequence: reverse   count   Expect  Zscore
AAAATTTA:TAAATTTT	835	    1737.66	-21.66
GATTAATA:TATTAATC	550	    1326.89	-21.33
AATTAATA:TATTAATT	929	    1839.72	-21.24
AATTAATC:GATTAATT	977	    1861.79	-20.51
ATAATTAA:TTAATTAT	1033    1926.49	-20.36
GAAATTTA:TAAATTTC	378 	1031.07	-20.34
CCGATCGC:GCGATCGG	1323	2284.74	-20.12
GCGATCGA:TCGATCGC	1221	2121.78	-19.56
ACGATCGC:GCGATCGT	1188	2035.31	-18.78
GTTTAAAA:TTTTAAAC	519	    1151.17	-18.63
AAATATTA:TAATATTT	738	    1444.79	-18.60
AAAATTTG:CAAATTTT	857	    1591.95	-18.42
CATTAATC:GATTAATG	479	    1037.49	-17.34
ATATATAA:TTATATAT	271	    736.19	-17.15
ATTTAAAG:CTTTAAAT	419	    941.16	-17.02
AAATATTG:CAATATTT	687	    1287.74	-16.74
CAAATTTA:TAAATTTG	670	    1265.24	-16.74
CTTTAAAC:GTTTAAAG	468	    995.13	-16.71

This cell is adapted from Dr. Bernicks derivation provided in class
# Null Model derivation
If we assume complete independence of the any of the characters that make up a sequence K, we could approximate the expected count of K, E(K),  as:
$\begin{matrix} 
K=k_{1}k_{2}k_{3}k_{4}, Pr(k_{i})=\frac{c(k_{i})}{N} \\ 
E(K)=N * Pr(K) = N * 
Pr(k_{1})
Pr(k_{2})
Pr(k_{3})
Pr(k_{4}) 
\end{matrix} 
$
The previous statement outlines a Markov(0) approximation for the expected count (E) of our sequence K. 

This assumption of independence is often inadequate due to factors such as codon bias influencing the selection of 3-mers based on underlying amino acid frequencies. To address this, we employ a Markov model, where the probability of any symbol is dependent on the preceding characters. For a k-mer of size 4, we condition our character probabilities using the preceding two bases, obviating the need for the count of the specific 4-mer. Edge conditions are essential to consider in our approximation. Generally, we can utilize a model of order k-2, where k is the size of the k-mer, as long as we possess sufficient data. 

$
\begin{matrix} 
Pr(K)=\frac{E(K)}{N} = 
Pr(k_{1})
Pr(k_{2}|k_{1})
Pr(k_{3}|k_{1}k_{2})
Pr(k_{4}|k_{2}k_{3}) \\
= \frac{c(k_{1})}{N}
\frac{c(k_{1}k_{2})}{c(k_{1})}
\frac{c(k_{1} k_{2} k_{3})} {c(k_{1} k_{2})}
\frac{c(k_{2} k_{3} k_{4})} {c(k_{2} k_{3})} \\
= \frac{1}{N}
\frac{c(k_{1} k_{2} k_{3})c(k_{2} k_{3} k_{4})} {c(k_{2} k_{3})}
\end{matrix}
$


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Oct  4 17:22:02 2023
@author: jlarbale
"""

import sys
from itertools import product
import math

# FastAreader by Dr. David Bernick
class FastAreader:
    def __init__(self, fname=''):
        '''contructor: saves attribute fname '''

        self.fname = fname
        self.fileH = None

    def doOpen(self):
        if self.fname == '':
            return sys.stdin
        else:
            return open(self.fname)

    def readFasta(self):

        header = ''
        sequence = ''

        with self.doOpen() as self.fileH:

            header = ''
            sequence = ''

            # skip to first fasta header
            line = self.fileH.readline()
            while not line.startswith('>'):
                line = self.fileH.readline()
            header = line[1:].rstrip()

            for line in self.fileH:
                if line.startswith('>'):
                    yield header, sequence
                    header = line[1:].rstrip()
                    sequence = ''
                else:
                    sequence += ''.join(line.rstrip().split()).upper()

        yield header, sequence


class CommandLine():
    '''
    Handle the command line, usage and help requests.

    CommandLine uses argparse, now standard in 2.7 and beyond. 
    it implements a standard command line argument parser with various argument options,
    a standard usage and help, and an error termination mechanism do-usage_and_die.

    attributes:
    all arguments received from the commandline using .add_argument will be
    avalable within the .args attribute of object instantiated from CommandLine.
    For example, if myCommandLine is an object of the class, and requiredbool was
    set as an option using add_argument, then myCommandLine.args.requiredbool will
    name that option.

    '''

    def __init__(self, inOpts=None):
        '''
        CommandLine constructor.
        Implements a parser to interpret the command line argv string using argparse.
        '''

        import argparse
        self.parser = argparse.ArgumentParser(
            description='Program prolog - a brief description of what this thing does',
            epilog='Program epilog - some other stuff you feel compelled to say',
            add_help=True,  # default is True
            prefix_chars='-',
            usage='%(prog)s [options] -option1[default] <input >output'
        )

        self.parser.add_argument('-l', '--minMotif', nargs='?', default=1, action='store',
                                 help='min kMer size ')
        self.parser.add_argument('-m', '--maxMotif', nargs='?', default=8, action='store',
                                 help='max kMer size ')
        self.parser.add_argument('-c', '--cutoff', nargs='?', type=float, default=.01, action='store',
                                 help='Zscore cutoff')

        self.parser.add_argument(
            '-v', '--version', action='version', version='%(prog)s 0.1')
        if inOpts is None:
            self.args = self.parser.parse_args()
        else:
            self.args = self.parser.parse_args(inOpts)


def revcomp(kmer):
    transDic = {'A': 'T',
                'T': 'A', 
                'G': 'C', 
                'C': 'G'}
    return "".join([transDic[i] for i in kmer[::-1]])


def key(kmer):
    exp = [kmer, revcomp(kmer)] #expand,
    return sorted(exp)[0]       #collapse for alphabetical leader


class Genome:
    '''
    reads a fasta file and ranks motifs based on how statistically underrepresented the specific motif is. 
    We need only consider sequence motifs that are up to about 8 in length ( this is a program option). In 
    order to use the equation that we developed in class, the minimum size should be at least 3 ( this is an 
    option also). We will find that there are quite a few of these, so we need to specify a statistical cutoff. 
    For this assignment, we will use Z-scores. The scores we are looking for will be negative since we are 
    looking for underrepresented sequences (this cutoff is a program option too). Finally, we need to sort 
    the output by decreasing motif size and then increasing (most negative to least negative) z-score within 
    motif size groups and print it out.
    '''
    def __init__(self, kMin, kMax, psudo=0):
        self.psudo = psudo
        self.kMin = int(kMin)
        self.kMax = int(kMax)
        self.kMers = []  # reference kmers
        self.kCounts = {}
        self.genomicSum = 0


        for k in range(self.kMin-2, self.kMax+1):  # inclusivity?
            kmerCombinations = product('ACGT', repeat=k) # Generate all possible k-mers of length k using 'ACGT'
            self.kMers.extend([''.join(kmer) for kmer in kmerCombinations])

        for kMer in self.kMers:
            countKey = key(kMer)
            self.kCounts[countKey] = self.psudo


    def countKmers(self, sequence):
        ''' 
        Recieves genome 1 record at a time. Populate class attribute countsK with counts of each k-mer.
        '''
        self.genomicSum += len(sequence) 
        
        for i in range(len(sequence)):
            for l in range(self.kMin-2, self.kMax+1):
                kMer = sequence[i: i + l]
                countKey = key(kMer)
                self.kCounts[countKey] += 1


    def calcE(self):
        '''
        calculate expected counts based on Markov(k-2) derived relationship
        '''
        results = []
        ref = []

        for kMer in self.kMers:
            if len(kMer)>self.kMin-1:
                try:
                    headCount = self.kCounts[key(kMer[:-1])]
                    tailCount = self.kCounts[key(kMer[1:])]
                    bodyCount = self.kCounts[key(kMer[1:-1])]
                    e = (headCount * tailCount) / bodyCount  # mu
                except ZeroDivisionError:
                    e = 0

                stddev = math.sqrt((e*1-e/self.genomicSum))
                actual = self.kCounts[key(kMer)]
                zscore = (actual-e)/(stddev) if stddev > 0 else 0
                
                if key(kMer) not in ref:
                    results.append([key(kMer), actual, e, zscore])
                ref.append(key(kMer))

        sortedResults = sorted(results, key=lambda x: (len(x[0]), -x[3], x[0]))
        return sortedResults[::-1]

import argparse
def main():
    parser = argparse.ArgumentParser(description="Find underrepresented motifs in a genome.")
    parser.add_argument("--minMotif", type=int, required=True, help="Minimum motif size")
    parser.add_argument("--maxMotif", type=int, required=True, help="Maximum motif size")
    parser.add_argument("--cutoff", type=float, required=True, help="Cutoff value")
    parser.add_argument("--input", type=str, required=True, help="Input file")

   
    args = parser.parse_args()


    thisGenome = Genome(args.minMotif, args.maxMotif)

    fasta_reader = FastAreader(args.input)
    for head, seq in fasta_reader.readFasta():
        thisGenome.countKmers(seq)

    result = thisGenome.calcE()

    
    for item in result:
        pVal = item[3]
        if pVal <= args.cutoff:
            seq = item[0]
            rSeq = revcomp(item[0])
            count = item[1]
            E = item[2]
            print('{0:8}:{1:8}\t{2:0d}\t{3:0.2f}\t{4:0.2f}'.format(seq, rSeq, count, E, pVal))

if __name__ == "__main__":
    main()

In [ ]:
Ex: missingMotif.py --minMotif=3 --maxMotif=8 --cutoff=-4.0 < [input file] > [output file]